<a href="https://colab.research.google.com/github/CHOIHANJOON/Another-Day-Another-Planting/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy
from konlpy.tag import Okt
okt=Okt()
def build_bag_of_words(document):
  document = document.replace('.','')
  tokenized_document = okt.morphs(document)
  word_to_index = {}
  bow = []
  for word in tokenized_document:
    if word not in word_to_index.keys():
      word_to_index[word] = len(word_to_index)
      bow.insert(len(word_to_index) - 1,1)
    else:
      index = word_to_index.get(word)
      bow[index] = bow[index] + 1
  return word_to_index, bow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 4.6 MB/s 
     |████████████████████████████████| 453 kB 49.5 MB/s 


In [2]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
vocab, bow = build_bag_of_words(doc1)
print('vocabulary :', vocab)
print('bag of words vector :', bow)
doc2 = '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'
vocab, bow = build_bag_of_words(doc2)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

doc3 = doc1 + '' + doc2
vocab, bow = build_bag_of_words(doc3)
print('vocabulary:', vocab)
print('bag of words vector :', bow)

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
bag of words vector : [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]
vocabulary : {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
bag of words vector : [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]
vocabulary: {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}
bag of words vector : [1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = ['you know I want your love. because I love you']
vector = CountVectorizer()
print('bag of words vector :', vector.fit_transform(corpus).toarray())
print('vocabulary :', vector.vocabulary_)

bag of words vector : [[1 1 2 1 2 1]]
vocabulary : {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=["the","a","an","is", "not"])
print('bag of words vector :', vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)
vect = CountVectorizer(stop_words="english")
print('bag of words vector :', vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)
import nltk
nltk.download('stopwords')
nltk_stop_words = stopwords.words("english")
vect = CountVectorizer(stop_words=nltk_stop_words)
print('bag of words vector :', vect.fit_transform(text).toarray())
print('vocabulary :', vect.vocabulary_)

bag of words vector : [[1 1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}
bag of words vector : [[1 1 1]]
vocabulary : {'family': 0, 'important': 1, 'thing': 2}
bag of words vector : [[1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
import pandas as pd # 데이터프레임 사용을 위해
from math import log # IDF 계산을 위해

docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [6]:
N = len(docs)
def tf(t, d):
  return d.count(t)
def idf(t):
  df = 0

  for doc in docs:
    df += t in doc
  return log(N/(df+1))
def tfidf(t,d):
  return tf(t,d)*idf(t)

result = []

for i in range(N):
  result.append([])
  d=docs[i]
  for j in range(len(vocab)):
    t= vocab[j]
    result[-1].append(tf(t,d))

tf_ = pd.DataFrame(result,columns = vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [7]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [8]:
result = []

for i in range(N):
  result.append([])
  d=docs[i]
  for j in range(len(vocab)):
    t= vocab[j]
    result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result,columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus =['you know I want your love', 'I like you', 'what should I do',]

tfidfv = TfidfVectorizer()
#코퍼스로부터 각 단어의 빈도수를 기록
print(tfidfv.fit_transform(corpus).toarray())
#각 단어와 맵핑된 인덱스 출력
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [10]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
def cos_sim(A,B):
  return dot(A,B)/(norm(A)*norm(B))
doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

print('문서 1과 문서2의 유사도 :', cos_sim(doc1, doc2))
print('문서 1과 문서3의 유사도 :', cos_sim(doc1, doc3))
print('문서 2와 문서3의 유사도 :', cos_sim(doc2, doc3))

문서 1과 문서2의 유사도 : 0.6666666666666667
문서 1과 문서3의 유사도 : 0.6666666666666667
문서 2와 문서3의 유사도 : 1.0000000000000002


In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#파일 읽고 상위 2개 행만 보기
df = pd.read_csv('movies_metadata.csv')
df.head(2)
#상위 2만개의 샘플을 data에 저장
data = df.head(20000)
# overview 열에 존재하는 모든 결측값 수 확인
data['overview'].isnull().sum()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


135

In [13]:
#결측값을 빈 값으로 대체
data['overview'] = data['overview'].fillna('')
tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기 (shape) :', tfidf_matrix.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


TF-IDF 행렬의 크기 (shape) : (20000, 47487)


In [14]:
#영화의 타이틀을 사용해 해당 영화의 인덱스를 받아온다.
#해당 영화와 모든 영화와의 유사도를 가져온다.
#유사도에 따라 영화들을 정렬한다.
#가장 유사한 10개의 영화를 받아온다.
#가장 유사한 10개의 영화의 인덱스를 얻는다.
#가장 유사한 10개의 영화의 제목을 리턴한다.


In [15]:
data.title[0]
data[data['title'] == 'Toy Story'].index[0]

0

In [16]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix) 
# 20000개의 문서 벡터에 대해서 상호 간의 코사인 유사도 확인 print('코사인 유사도 연산 결과 :',cosine_sim.shape)

In [17]:
def get_recommendations(title, sim = cosine_sim):
  idx = data[data['title'] == title].index[0]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
  sim_scores = sim_scores[1:11]
  movie_ind = [i[0] for i in sim_scores]
  return data.title.iloc[movie_ind]

In [18]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object